---
# Praxisteil 4 - Metaheuristiken
---

---
## 1. Iterated Greedy
---

---
### 1.1 Wiederholung allgemeiner Ablauf
---

Ablauf Iterated Greedy

![AblaufIG](../DataFiles/IteratedGreedyAblauf.png)

----
### 1.2 Startlösung
----
NEH-Heuristik

In [1]:
from Solver import *

data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json")

solver = Solver(data, 1060) #1014, 754 |  1060, 736

startSolution = solver.ConstructionPhase("NEH")

currentSolution = deepcopy(startSolution)


Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752


----
### 1.3 Destruktion
----
Entferne zufällig _numberJobsToRemove_ Aufträge aus aktueller Lösung. Dabei ist _numberJobsToRemove_ ein Parameter des Iterated Greedy.
Als Ergebnis ensteht die Menge entfernter Aufträge _removedJobs_ und eine unvollständige Lösung _partialPermutation_.

In [2]:
numberJobsToRemove = 2 # 1st parameter

def Destruction(solution):
    removedJobs = solver.RNG.choice(solver.InputData.n, size = numberJobsToRemove, replace = False).tolist()
    partialPermutation = [i for i in solution.Permutation if i not in removedJobs]
    return removedJobs, partialPermutation

rj, pp = Destruction(currentSolution)

----
### 1.4 Konstruktion
----
Füge die _removedJobs_ unter Beachtung der Reihenfolge an der jeweils besten Position (NEH) wieder zu _partialPermutation_ hinzu und gib die neue vollständige Lösung zurück. 

In [3]:
def Construction(removedJobs, partialPermutation):
    completeSolution = Solution(solver.InputData.InputJobs, partialPermutation)
    for job in removedJobs:
        solver.EvaluationLogic.DetermineBestInsertion(completeSolution, job)

    return completeSolution

newSolution = Construction(rj, pp)

----
### 1.5 Akzeptanz und Gedächtnis
---- 
Akzeptiere neue Lösung als aktuelle Lösung, wenn
* neue Lösung besser als aktuelle Lösung (immer)
* Akzeptanzkriterium erfüllt (zufällig)

Akzeptanzkriterium bei Ruiz/ Stützle:  $\pi''$ - neue Lösung (_newSolution_), $\pi$ - aktuelle Lösung (_currentSolution_)
$$
    \begin{equation}
    e^{- \frac{C_{\max}\left( \pi'' \right) - C_{\max}\left( \pi \right)}{Temperature}} \text{ mit } Temperature = T \cdot \frac{\sum Bearbeitungszeiten}{n \cdot m \cdot 10}
    \end{equation}
$$  
$T$ ist der 2. Parameter des Iterated Greedy. Je höher $T$ ist, desto größer ist die Wahrscheinlichkeit, schlechtere Lösungen zu akzeptieren.

Neue beste Lösung wird im _SolutionPool_ gespeichert.

In [4]:
def AcceptWorseSolution(currentObjectiveValue, newObjectiveValue):  
    temperature = baseTemperature * solver.InputData.TotalProcessingTime / (solver.InputData.n * solver.InputData.m * 10)
    probability = math.exp(-(newObjectiveValue - currentObjectiveValue) / temperature)

    return solver.RNG.random() <= probability

In [5]:
print(f'Solution before checking for acceptance: \n{currentSolution}')

baseTemperature = 1 # 2nd parameter
currentBest = solver.SolutionPool.GetLowestMakespanSolution().Makespan

if newSolution.Makespan < currentSolution.Makespan:
    currentSolution = newSolution
    if newSolution.Makespan < currentBest:
        solver.SolutionPool.AddSolution(newSolution)
        currentBest = newSolution.Makespan
else:
    if AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
        solver.SolutionPool.AddSolution(newSolution)
        currentBest = newSolution.Makespan

        
print(f'Solution after checking for acceptance: \n{currentSolution}')

Solution before checking for acceptance: 
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752
Solution after checking for acceptance: 
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736


----
### 1.6 Lokale Suche
----
Verfahren der lokalen Suche können optional verwendet werden, um die aktuelle Lösung zu verbessern. Häufig wird __Iterative Improvement__ mit der Insertion-Nachbarschaft verwendet.

In [6]:
print(f'Solution before Local Search:\n{currentSolution}')

localSearch = IterativeImprovement(solver.InputData, "BestImprovement", ["Insertion"])
localSearch.Initialize(solver.EvaluationLogic, solver.SolutionPool)
currentSolution = localSearch.Run(currentSolution)

print(f'Solution after nach Local Search\n{currentSolution}')

Solution before Local Search:
The permutation [4, 3, 0, 7, 1, 8, 2, 9, 5, 6] results in a Makespan of 736
Solution after nach Local Search
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713


----
### 1.7 Iterated Greedy als Steuerungsmechanismus der Komponenten
----
Iterated Greedy ist iterative Abfolge von Destruktion und Konstruktion $\Rightarrow$ wird in Schleife immer wieder durchgeführt bis Stoppkriterium erreicht ist. 

Hier im Beispiel ist Stoppkriterium die Anzahl an Iterationen _maxIterations_, aber bspw. auch Zeitlimit oder Anzahl Iterationen ohne Verbesserungen möglich.

In [12]:
maxIterations = 10 # stop criterion number of iterations

def Run(currentSolution):
    localSearch = IterativeImprovement(solver.InputData, "BestImprovement", ["Insertion"])
    localSearch.Initialize(solver.EvaluationLogic, solver.SolutionPool)
    currentSolution = localSearch.Run(currentSolution)
    currentBest = solver.SolutionPool.GetLowestMakespanSolution().Makespan
    for i in range(maxIterations):
        rj, pp = Destruction(currentSolution)
        newSolution = Construction(rj, pp)

        if newSolution.Makespan < currentSolution.Makespan:
            currentSolution = newSolution
            if newSolution.Makespan < currentBest:
                solver.SolutionPool.AddSolution(newSolution)
                currentBest = newSolution.Makespan
            else:
                if AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
                    solver.SolutionPool.AddSolution(newSolution)
                    currentBest = newSolution.Makespan
        
        currentSolution = localSearch.Run(currentSolution)

    return currentSolution

Iterated Greedy ausführen

In [13]:
solver = Solver(data, 1060)

startSolution = solver.ConstructionPhase("NEH")

bestSolution = Run(startSolution)

print(f'Best solution found.\n{bestSolution}')

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752
Best solution found.
The permutation [1, 8, 4, 3, 0, 5, 2, 7, 9, 6] results in a Makespan of 713


---
## 2. Klasse für Iterated Greedy
---

Analog wie für den Algorithmus _IterativeImprovement_ soll jetzt auch für _IteratedGreedy_ eine eigene Klasse angelegt werden, damit eine Instanz der Klasse an den Solver übergeben werden kann.

_Iterated Greedy_ soll wie _IterativeImprovement_ von _ImprovementAlgorithm_ erben.

Notwendige __Parameter__ sind Attribute:
* NumberJobsToRemove
* BaseTemperature
* MaxIterations
* Instanz des Local Search Algorithmus

EvaluationLogic, SolutionPool und random number generator werden von Solver an Algorithmus übergeben.

__Programmieraufgabe:__ Ergänzen sie die Klasse _IteratedGreedy_, sodass sie an den Solver als Algorithmus übergeben werden kann.

__Funktionen:__ Konstruktor, Initialize, Destruction, Construction, AcceptWorseSolution, Run

In [16]:
class IteratedGreedy(ImprovementAlgorithm):
    def __init__(self, inputData, numberJobsToRemove, baseTemperature, maxIterations, localSearchAlgorithm = None):
        super().__init__(inputData)

        self.NumberJobsToRemove = numberJobsToRemove
        self.BaseTemperature = baseTemperature
        self.MaxIterations = maxIterations

        if localSearchAlgorithm is not None:
            self.LocalSearchAlgorithm = localSearchAlgorithm
        else:
            self.LocalSearchAlgorithm = IterativeImprovement(self.InputData, neighborhoodTypes=[]) # IterativeImprovement without a neighborhood does not modify the solution
    
    def Initialize(self, evaluationLogic, solutionPool, rng):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

        self.LocalSearchAlgorithm.Initialize(self.EvaluationLogic, self.SolutionPool)
    
    def Destruction(self, currentSolution):
        removedJobs = self.RNG.choice(self.InputData.n, size=self.NumberJobsToRemove, replace = False).tolist()

        partialPermutation = [i for i in currentSolution.Permutation if i not in removedJobs]

        return removedJobs, partialPermutation

    def Construction(self, removedJobs, permutation):
        completeSolution = Solution(self.InputData.InputJobs, permutation)
        for i in removedJobs:
            self.EvaluationLogic.DetermineBestInsertionAccelerated(completeSolution, i)

        return completeSolution

    def AcceptWorseSolution(self, currentObjectiveValue, newObjectiveValue):
        randomNumber = self.RNG.random()

        totalProcessingTime = sum(x.ProcessingTime(i) for x in self.InputData.InputJobs for i in range(len(x.Operations)))
        Temperature = self.BaseTemperature * totalProcessingTime / (self.InputData.n * self.InputData.m * 10)
        probability = math.exp(-(newObjectiveValue - currentObjectiveValue) / Temperature)
        
        return randomNumber <= probability

    def Run(self, currentSolution):
        currentSolution = self.LocalSearchAlgorithm.Run(currentSolution)

        currentBest = self.SolutionPool.GetLowestMakespanSolution().Makespan
        iteration = 0
        while(iteration < self.MaxIterations):
            removedJobs, partialPermutation = self.Destruction(currentSolution)
            newSolution = self.Construction(removedJobs, partialPermutation)

            newSolution = self.LocalSearchAlgorithm.Run(newSolution)
            
            if newSolution.Makespan < currentSolution.Makespan:
                currentSolution = newSolution

                if newSolution.Makespan < currentBest:
                    print(f'New best solution in iteration {iteration}: {currentSolution}')
                    self.SolutionPool.AddSolution(currentSolution)
                    currentBest = newSolution.Makespan

            elif self.AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
                currentSolution = newSolution

            iteration += 1

        return self.SolutionPool.GetLowestMakespanSolution()

Der Algorithmus kann jetzt in _RunLocalSearch()_ an den Solver übergeben und ausgeführt werden.

In [17]:
    data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json") #"../TestInstancesJson/Small/VFR40_10_3_SIST.json"

    insertion = IterativeImprovement(data, 'BestImprovement', ['Insertion'])
    taillardInsertion = IterativeImprovement(data, 'BestImprovement', ['TaillardInsertion'])
    iteratedGreedy = IteratedGreedy(
        data, 
        numberJobsToRemove=2, 
        baseTemperature=1, 
        maxIterations=100)

    solver = Solver(data, 1010)

    solver.RunLocalSearch(
        constructiveSolutionMethod='NEH',
        algorithm=iteratedGreedy)

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [4, 3, 7, 1, 8, 2, 9, 0, 5, 6] results in a Makespan of 752
New best solution in iteration 0: The permutation [4, 3, 0, 7, 1, 8, 2, 6, 9, 5] results in a Makespan of 736
New best solution in iteration 2: The permutation [8, 4, 3, 0, 7, 1, 2, 6, 9, 5] results in a Makespan of 726
New best solution in iteration 7: The permutation [4, 8, 5, 0, 3, 7, 1, 2, 6, 9] results in a Makespan of 724
New best solution in iteration 34: The permutation [1, 3, 8, 4, 0, 5, 2, 7, 6, 9] results in a Makespan of 722
New best solution in iteration 35: The permutation [1, 8, 3, 4, 0, 5, 2, 7, 6, 9] results in a Makespan of 713
Best found Solution.
The permutation [1, 8, 3, 4, 0, 5, 2, 7, 6, 9] results in a Makespan of 713


----
## 3. Exkurs: DEAP und Genetic Algorithms
----

Distributed Evolutionary Algorithms in Python ([DEAP](https://github.com/DEAP/deap)) ist ein Framework für Evolutionary Computation, mit u.a. den populationsbasierten Metaheuristiken Genetic Algorithms und Particle Swarm Optimization.

![Metaheuristiken](../DataFiles/MH.png)

### Wichtige Begriffe für Genetic Algorithms:
* Individuum: Einzelne Lösung
* Population: Sammlung von Individuen
* Generation: Population in einer Iteration
* Reproduktion: Erzeugung neuer Individuen aus bekannten Individuen
* Selektion: Auswahl von Individuen für Reproduktion und neue Generation
* Fitness: Bewertung eines Individuums (Zielfunktionswert)
* Mutation: Zufällige Veränderung eines Individuums

![GeneticAlgorithm](../DataFiles/GA.png)

----
### Beispiel DEAP
----

__Creator__

Meta-Factory um Klassen zu erzeugen

Parameters:
* name – The name of the class to create.
* base – A base class from which to inherit.
* attribute – One or more attributes to add on instantiation of this class, optional.


In [18]:
from Solver import *

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json")

solver = Solver(data, 1011)

creator.create("FitnessMin", base.Fitness, weights=[-1.0]) # minimization problem, weights as list
creator.create("Individual", list, fitness=creator.FitnessMin) # single individual stored in python list

__Toolbox__

Erzeugt Konfiguration des Algorithmus

In [19]:
toolbox = base.Toolbox() 

# Individual is a permutation of integer indices
toolbox.register("indices", solver.RNG.choice, range(solver.InputData.n), solver.InputData.n, replace=False)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

# Population is a collection of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
pop = toolbox.population(100) # 100 individuals in population

# Operators
toolbox.register("mate", tools.cxPartialyMatched) # set crossover 
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # set mutation
toolbox.register("select", tools.selTournament, tournsize=3) # set selection mechanism

__Fitness-Funktion__

Bewertung eines Indivduums

In [20]:
def EvalPFSP(solver, individual):
    solution = Solution(solver.InputData.InputJobs, individual)
    solver.EvaluationLogic.DefineStartEnd(solution)
    return [solution.Makespan]

toolbox.register("evaluate", EvalPFSP, solver) # fitness function

__Statistiken__

Sammlung von Werten während Laufzeit des Algorithmus'

In [21]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

__Hall of fame__

Gedächtnis zum Speichern der besten Lösungen; hier nur beste Lösung

In [22]:
hof = tools.HallOfFame(1)

__Ausführen__

In [23]:
random.seed(solver.Seed)
algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=0.8, mutpb=0.2, ngen=100, stats=stats, halloffame=hof)

bestPermutation = hof[0]
bestSolution = Solution(solver.InputData.InputJobs, bestPermutation)
solver.EvaluationLogic.DefineStartEnd(bestSolution)
print(f'Best found Solution.\n {bestSolution}')

gen	nevals	avg   	std    	min	max
0  	100   	866.87	42.9324	757	985
1  	81    	829.45	42.7935	730	954
2  	82    	820.48	40.6065	743	915
3  	91    	809.32	36.6756	743	900
4  	81    	793.49	34.1264	743	894
5  	78    	784.84	36.3518	739	895
6  	85    	778.38	38.1512	738	921
7  	76    	768.52	30.2561	738	879
8  	90    	764.48	25.6743	738	871
9  	84    	769.89	31.5598	739	857
10 	84    	772.37	40.845 	736	950
11 	83    	766.35	34.4184	736	879
12 	89    	762.21	35.8484	726	952
13 	77    	757.56	30.777 	726	868
14 	90    	756.72	26.8652	726	840
15 	80    	757.71	30.4999	726	868
16 	88    	760.39	42.3719	726	1002
17 	82    	747.93	25.1862	726	851 
18 	90    	746.3 	29.4607	726	911 
19 	77    	737.24	15.9004	726	824 
20 	90    	732.04	10.3884	726	778 
21 	84    	730.92	17.4159	726	831 
22 	81    	731.35	18.3174	726	816 
23 	80    	728.35	11.0258	726	792 
24 	90    	728.12	12.4758	726	840 
25 	86    	728.48	13.3877	726	840 
26 	87    	728.66	11.2483	726	816 
27 	82    	730.1 	16.0253	726	819 
28

----
#### Funktion für Solver
----

Parameter als Argumente
* Populationsgröße _populationSize_
* Anzahl Generationen _generations_
* Rekombinationswahrscheinlichkeit _matingProb_
* Mutationswahrscheinlichkeit _mutationProb_

In [24]:
def EvalPFSP(self, individual):
    solution = Solution(self.InputData.InputJobs, individual)
    self.EvaluationLogic.DefineStartEnd(solution)
    return [solution.Makespan]

def RunGeneticAlgorithm(self, populationSize, generations, matingProb, mutationProb):
    # Creator - meta-factory to create new classes
    creator.create("FitnessMin", base.Fitness, weights=[-1.0])
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox() 

    # Individual is a permutation of integer indices
    toolbox.register("indices", solver.RNG.choice, range(self.InputData.n), self.InputData.n, replace=False)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

    # Population is a collection of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(populationSize) # number of individuals in population

    # Operators
    toolbox.register("mate", tools.cxPartialyMatched) # set crossover 
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # set mutation
    toolbox.register("select", tools.selTournament, tournsize=3) # set selection mechanism
    
    # Fitness function
    toolbox.register("evaluate", self.EvalPFSP) 
    
    # Statistics during run time
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
            
    # Hall of fame --> Best individual
    hof = tools.HallOfFame(1)

    random.seed(self.Seed)
    algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=matingProb, mutpb=mutationProb, ngen=generations, stats=stats, halloffame=hof)

    bestPermutation = hof[0]
    bestSolution = Solution(self.InputData.InputJobs, bestPermutation)
    self.EvaluationLogic.DefineStartEnd(bestSolution)
    print(f'Best found Solution.\n {bestSolution}')

Solver.EvalPFSP = EvalPFSP
Solver.RunGeneticAlgorithm = RunGeneticAlgorithm

Vergleich Iterated Greedy und Genetic Algorithm

In [25]:
data = InputData("../TestInstancesJson/Large/VFR100_20_1_SIST.json") # TestInstances/Small/VFR40_10_3_SIST.txt 

taillardInsertion = IterativeImprovement(data, 'FirstImprovement', neighborhoodTypes=['TaillardInsertion'])
iteratedGreedy = IteratedGreedy(
    data, 
    numberJobsToRemove=2, 
    baseTemperature=1, 
    maxIterations=50, 
    localSearchAlgorithm=taillardInsertion)

solver = Solver(data, 1010)

solver.RunLocalSearch(
    constructiveSolutionMethod='NEH',
    algorithm=iteratedGreedy)
    
solver.RunGeneticAlgorithm(
    populationSize=100, 
    generations=200, 
    matingProb=0.8, 
    mutationProb=0.2)


Generating an initial solution according to NEH.
Constructive solution found.
The permutation [61, 80, 51, 99, 11, 8, 62, 41, 24, 94, 67, 63, 0, 27, 31, 30, 82, 52, 91, 1, 19, 59, 89, 4, 49, 88, 25, 47, 85, 81, 38, 23, 5, 56, 71, 93, 83, 2, 64, 33, 96, 20, 21, 77, 78, 13, 60, 74, 98, 90, 6, 76, 9, 3, 42, 84, 50, 86, 95, 72, 97, 35, 40, 92, 15, 39, 10, 68, 43, 17, 26, 36, 18, 57, 44, 29, 54, 79, 55, 22, 48, 53, 75, 16, 73, 12, 58, 28, 32, 87, 70, 34, 7, 66, 45, 69, 14, 65, 46, 37] results in a Makespan of 6596
New best solution in iteration 2: The permutation [51, 61, 28, 80, 8, 62, 99, 31, 52, 63, 11, 94, 41, 67, 1, 82, 30, 91, 24, 4, 59, 47, 49, 88, 81, 85, 25, 38, 27, 23, 89, 71, 5, 56, 93, 83, 33, 96, 20, 77, 64, 17, 13, 19, 78, 60, 98, 90, 21, 6, 76, 43, 9, 3, 84, 86, 95, 40, 10, 35, 42, 74, 92, 15, 39, 97, 68, 26, 36, 18, 57, 0, 44, 29, 50, 54, 79, 55, 22, 48, 53, 75, 16, 73, 12, 58, 32, 87, 70, 34, 7, 66, 72, 45, 69, 14, 2, 65, 46, 37] results in a Makespan of 6424
New best solut

/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	std    	min 	max 
0  	100   	7844.37	143.006	7500	8126
1  	78    	7767.61	149.492	7452	8199
2  	78    	7707.43	113.262	7346	7997
3  	76    	7689.86	134.223	7418	8138
4  	77    	7675.46	141.889	7425	8105
5  	90    	7666.71	141.713	7420	8145
6  	78    	7632.73	136.821	7411	8118
7  	86    	7605.7 	137.468	7371	7964
8  	82    	7596.6 	149.14 	7278	8035
9  	76    	7569.05	125.122	7278	7907
10 	81    	7549.81	136.695	7300	8057
11 	88    	7573.88	153.392	7262	8111
12 	87    	7535.16	135.361	7262	7859
13 	86    	7519.79	138.436	7221	7923
14 	87    	7531.76	157.896	7288	8240
15 	86    	7513.31	148.557	7270	7894
16 	85    	7466.2 	122.054	7243	7794
17 	71    	7480.18	135.351	7243	7901
18 	91    	7495.01	149.883	7256	7909
19 	85    	7493.16	149.326	7272	7939
20 	89    	7497.03	166.726	7263	8022
21 	86    	7471.71	141.29 	7216	7791
22 	92    	7471.26	134.687	7251	7797
23 	84    	7493.63	132.505	7256	7865
24 	93    	7491.55	134.574	7181	7903
25 	83    	7478.33	130.588	7195	7815
2